In [2]:
import graphene

In [3]:
from graphqlclient import GraphQLClient

In [7]:
import json

In [4]:
client = GraphQLClient('https://circleci.com/graphql-unstable')


## Discovering Objects

In [14]:
result = client.execute('''
{
  __schema {
    types {b
      name
    }
  }
}
''')
print(json.dumps(json.loads(result),indent=4, sort_keys=True))

{
    "data": {
        "__schema": {
            "types": [
                {
                    "name": "Account"
                },
                {
                    "name": "AccountConnection"
                },
                {
                    "name": "AccountEdge"
                },
                {
                    "name": "ActiveUserConnection"
                },
                {
                    "name": "AddToAllowedContextGroupsInput"
                },
                {
                    "name": "AddToAllowedContextGroupsPayload"
                },
                {
                    "name": "BillingDetailsInput"
                },
                {
                    "name": "BillingPeriod"
                },
                {
                    "name": "Boolean"
                },
                {
                    "name": "ConfigError"
                },
                {
                    "name": "Context"
                },
                {

## Discover Fields of an object

In [24]:
result = client.execute('''
{
  __type(name: "PlanMetrics") {
    name
    kind
    fields {
      name
      type {
        name
        kind
        ofType {
          name
          kind
        }
      }
    }
  }
}
''')
print(json.dumps(json.loads(result),indent=4, sort_keys=True))

{
    "data": {
        "__type": {
            "fields": [
                {
                    "name": "activeUsers",
                    "type": {
                        "kind": "NON_NULL",
                        "name": null,
                        "ofType": {
                            "kind": "OBJECT",
                            "name": "ActiveUserConnection"
                        }
                    }
                },
                {
                    "name": "byProject",
                    "type": {
                        "kind": "NON_NULL",
                        "name": null,
                        "ofType": {
                            "kind": "OBJECT",
                            "name": "UsageMetricByProjectConnection"
                        }
                    }
                },
                {
                    "name": "projects",
                    "type": {
                        "kind": "NON_NULL",
                        "name": null,


## Query Object Details

In [19]:

result = client.execute('''
{
  plan(orgId: "f22b6566-597d-46d5-ba74-99ef5bb3d85c") {
    metrics(dateRange: {start:"2018-01-01", end: "2018-12-03"}) {
      total{
        credits
      },
      projects{totalCount}
    }
  }
}
''')
print(json.dumps(json.loads(result),indent=4, sort_keys=True))

{
    "data": null,
    "errors": [
        {
            "arguments": {
                "orgId": "f22b6566-597d-46d5-ba74-99ef5bb3d85c"
            },
            "locations": [
                {
                    "column": 0,
                    "line": 2
                }
            ],
            "message": "Non-nullable field was null.",
            "query-path": [
                "plan"
            ]
        },
        {
            "arguments": {
                "orgId": "f22b6566-597d-46d5-ba74-99ef5bb3d85c"
            },
            "locations": [
                {
                    "column": 0,
                    "line": 2
                }
            ],
            "message": "Something unexpected happened.",
            "query-path": [
                "plan"
            ],
            "type": "INTERNAL"
        }
    ]
}


## Authentication
`Non-nullable field was null.` == no auth token found 

In [22]:
import os
client.inject_token(os.getenv('CIRCLECI_API_TOKEN'))

In [23]:

result = client.execute('''
{
  plan(orgId: "f22b6566-597d-46d5-ba74-99ef5bb3d85c") {
    metrics(dateRange: {start:"2018-01-01", end: "2018-12-03"}) {
      total{
        credits
      },
      projects{totalCount}
    }
  }
}
''')
print(json.dumps(json.loads(result),indent=4, sort_keys=True))

{
    "data": {
        "plan": {
            "metrics": {
                "projects": {
                    "totalCount": 245
                },
                "total": {
                    "credits": 10111001
                }
            }
        }
    }
}


## Discover nested objects
We saw above that `PlanMetrics` also exposes a `byProject` field of type `UsageMetricByProjectConnection`
                        

In [41]:

result = client.execute('''
{
  plan(orgId: "f22b6566-597d-46d5-ba74-99ef5bb3d85c") {
    metrics(dateRange: {start:"2018-01-01", end: "2018-12-03"}) {
      total{
        credits
      },
      projects{totalCount}
      byProjects
    }
  }
}
''')
print(json.dumps(json.loads(result),indent=4, sort_keys=True))

{
    "errors": [
        {
            "field": "byProjects",
            "locations": [
                {
                    "column": 64,
                    "line": 4
                }
            ],
            "message": "Cannot query field `byProjects' on type `PlanMetrics'.",
            "query-path": [
                "plan",
                "metrics"
            ],
            "type": "PlanMetrics"
        }
    ]
}


Let's try to understand that object more.

In [25]:
result = client.execute('''
{
  __type(name: "UsageMetricByProjectConnection") {
    name
    kind
    fields {
      name
      type {
        name
        kind
        ofType {
          name
          kind
        }
      }
    }
  }
}
''')
print(json.dumps(json.loads(result),indent=4, sort_keys=True))

{
    "data": {
        "__type": {
            "fields": [
                {
                    "name": "nodes",
                    "type": {
                        "kind": "NON_NULL",
                        "name": null,
                        "ofType": {
                            "kind": "LIST",
                            "name": null
                        }
                    }
                }
            ],
            "kind": "OBJECT",
            "name": "UsageMetricByProjectConnection"
        }
    }
}


Hmm, only  has `nodes` can we get those?

In [32]:

result = client.execute('''
{
  plan(orgId: "f22b6566-597d-46d5-ba74-99ef5bb3d85c") {
    metrics(dateRange: {start:"2018-01-01", end: "2018-12-03"}) {
      total{
        credits
      },
      projects{totalCount}
      byProject{nodes{id}}
    }
  }
}
''')
print(json.dumps(json.loads(result),indent=4, sort_keys=True))

{
    "errors": [
        {
            "field": "id",
            "locations": [
                {
                    "column": 21,
                    "line": 9
                }
            ],
            "message": "Cannot query field `id' on type `UsageMetricByProject'.",
            "query-path": [
                "plan",
                "metrics",
                "byProject",
                "nodes"
            ],
            "type": "UsageMetricByProject"
        }
    ]
}


No, let's dig into that object type more

In [33]:
result = client.execute('''
{
  __type(name: "UsageMetricByProject") {
    name
    kind
    fields {
      name
      type {
        name
        kind
        ofType {
          name
          kind
        }
      }
    }
  }
}
''')
print(json.dumps(json.loads(result),indent=4, sort_keys=True))

{
    "data": {
        "__type": {
            "fields": [
                {
                    "name": "aggregate",
                    "type": {
                        "kind": "NON_NULL",
                        "name": null,
                        "ofType": {
                            "kind": "OBJECT",
                            "name": "PlanMetric"
                        }
                    }
                },
                {
                    "name": "project",
                    "type": {
                        "kind": "NON_NULL",
                        "name": null,
                        "ofType": {
                            "kind": "OBJECT",
                            "name": "Project"
                        }
                    }
                }
            ],
            "kind": "OBJECT",
            "name": "UsageMetricByProject"
        }
    }
}


Ahh, 2 fields are aggregate and project, lets unfurl those!

In [37]:

result = client.execute('''
{
  plan(orgId: "f22b6566-597d-46d5-ba74-99ef5bb3d85c") {
    metrics(dateRange: {start:"2018-01-01", end: "2018-12-03"}) {
      total{
        credits
      },
      projects{totalCount}
      byProject{nodes{aggregate{credits}}}
    }
  }
}
''')
print(json.dumps(json.loads(result),indent=4, sort_keys=True))

{
    "data": {
        "plan": {
            "metrics": {
                "byProject": {
                    "nodes": [
                        {
                            "aggregate": {
                                "credits": 3483461
                            }
                        },
                        {
                            "aggregate": {
                                "credits": 2775963
                            }
                        },
                        {
                            "aggregate": {
                                "credits": 788085
                            }
                        },
                        {
                            "aggregate": {
                                "credits": 516314
                            }
                        },
                        {
                            "aggregate": {
                                "credits": 348269
                            }
                        

In [36]:
result = client.execute('''
{
  __type(name: "PlanMetric") {
    name
    kind
    fields {
      name
      type {
        name
        kind
        ofType {
          name
          kind
        }
      }
    }
  }
}
''')
print(json.dumps(json.loads(result),indent=4, sort_keys=True))

{
    "data": {
        "__type": {
            "fields": [
                {
                    "name": "credits",
                    "type": {
                        "kind": "NON_NULL",
                        "name": null,
                        "ofType": {
                            "kind": "SCALAR",
                            "name": "Int"
                        }
                    }
                },
                {
                    "name": "seconds",
                    "type": {
                        "kind": "NON_NULL",
                        "name": null,
                        "ofType": {
                            "kind": "SCALAR",
                            "name": "Int"
                        }
                    }
                }
            ],
            "kind": "OBJECT",
            "name": "PlanMetric"
        }
    }
}


## Summary
This query shows priject summary credits and name for the range specified.

In [39]:
result = client.execute('''
{
  plan(orgId: "f22b6566-597d-46d5-ba74-99ef5bb3d85c") {
    metrics(dateRange: {start:"2018-11-01", end: "2018-12-03"}) {
      total{
        credits
      },
      projects{totalCount}
      byProject{nodes{aggregate{credits},project{name}}}
    }
  }
}
''')
print(json.dumps(json.loads(result),indent=4, sort_keys=True))

{
    "data": {
        "plan": {
            "metrics": {
                "byProject": {
                    "nodes": [
                        {
                            "aggregate": {
                                "credits": 424653
                            },
                            "project": {
                                "name": "circleci-images"
                            }
                        },
                        {
                            "aggregate": {
                                "credits": 223240
                            },
                            "project": {
                                "name": "circle"
                            }
                        },
                        {
                            "aggregate": {
                                "credits": 72335
                            },
                            "project": {
                                "name": "server-manifest"
                            